In [1]:
# !conda install --yes numpy

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
!nvidia-smi

Thu Jan 19 09:06:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# %%capture
!pip3 install datasets
!pip3 install transformers
!pip3 install torchaudio
!pip3 install librosa
!pip3 install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.7 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: levenshtein
    Found existing installation: Levenshtein 0.20.9
    Uninstalling Levenshtein-0.20.9:
      Successfully uninstalled Levenshtein-0.20.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-levenshtein 0.20.9 requires Levenshtein==0.20.9, but you have levenshtein 0.20.2 which is incompatible.


In [6]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "ru", split="train+validation")
common_voice_test = load_dataset("common_voice", "ru", split="test")

Generating train split:   0%|          | 0/15481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8007 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7963 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/10247 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/74256 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/3056 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ru/6.1.0/7cd6a2cd99f885b3ec1205a6aee65d9b8c7b36a2c0f482fa4a1dde3d29860f21. Subsequent calls will reuse this data.


In [7]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [9]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [10]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/23444 [00:00<?, ?ex/s]

  0%|          | 0/8007 [00:00<?, ?ex/s]

In [11]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,мы ненадолго извините
1,конференция по разоружению утратила убедительность в качестве единого многостороннего форума по разоружению и нераспространению
2,темпы годового прироста вкладов населения в банках достигли минимального уровня за последние пять лет
3,но наш регион не может расти и двигаться вперед отдельно от других
4,наша приверженность борьбе против табака остается непоколебимой
5,как я вижу слова только что попросил посол мексики
6,мы принимаем к сведению ваши заключительные замечания
7,я кратко выскажу то о чем обязан сказать
8,я также хочу ответить на ложные беспочвенные и неправомерные утверждения в отношении этих островов
9,по мнению бельгии и нидерландов такая ситуация сохраняться больше не может


In [12]:
!mkdir pretrained
!wget -O model.zip "https://downloader.disk.yandex.ru/disk/d25e4558b7a19cbe9197bee41992d8dc24b5875267aab0bde69843a503eabe13/63c92b28/SAFtUcwKI0ZrxCUQ5i2Bv7CpBsn1qzqtgE63ze8uzDtIJdB8c2w6L9H5KASWJheUq9dPlaVh7iA1sMqQVp0-yA%3D%3D?uid=0&filename=model.zip&disposition=attachment&hash=wlsHaIodv6q7Duj95SGiolqQI7lxoMYKDd0yw9hwJMVDwNKUsjIXFtNiPUUC9FR0q/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=1482923029&fsize=344969172&hid=d513d22ec211e37069e3d7bc10b9599c&media_type=compressed&tknv=v2"
!unzip -o model.zip -d pretrained

--2023-01-19 09:13:43--  https://downloader.disk.yandex.ru/disk/d25e4558b7a19cbe9197bee41992d8dc24b5875267aab0bde69843a503eabe13/63c92b28/SAFtUcwKI0ZrxCUQ5i2Bv7CpBsn1qzqtgE63ze8uzDtIJdB8c2w6L9H5KASWJheUq9dPlaVh7iA1sMqQVp0-yA%3D%3D?uid=0&filename=model.zip&disposition=attachment&hash=wlsHaIodv6q7Duj95SGiolqQI7lxoMYKDd0yw9hwJMVDwNKUsjIXFtNiPUUC9FR0q/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=1482923029&fsize=344969172&hid=d513d22ec211e37069e3d7bc10b9599c&media_type=compressed&tknv=v2
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s340vla.storage.yandex.net/rdisk/d25e4558b7a19cbe9197bee41992d8dc24b5875267aab0bde69843a503eabe13/63c92b28/SAFtUcwKI0ZrxCUQ5i2Bv7CpBsn1qzqtgE63ze8uzDtIJdB8c2w6L9H5KASWJheUq9dPlaVh7iA1sMqQVp0-yA==?uid=0&filename=mode

In [14]:
import json


vocab_dict1 = {"m": 0, "\u0451": 1, "e": 2, "i": 3, "\u0436": 4, "\u2011": 5, "s": 6, "\u044f": 7, "\u0440": 8, "\u043c": 9, "\u043d": 10, "\u00ab": 11, "\u0439": 12, "g": 13, "\u0442": 14, "\u2013": 15, "k": 16, "z": 17, "\u2014": 18, "\u0437": 19, "'": 20, "a": 21, "\u0434": 22, "\u043b": 23, "\u00bb": 24, "\u0447": 25, "\u0441": 26, "\u0431": 27, "h": 28, "c": 29, "(": 30, "\u0438": 31, "l": 32, "\u0449": 33, "\u0444": 34, "o": 35, "\u0448": 36, "\u0443": 37, "\u0445": 38, "\u0433": 39, "\u0446": 40, "\u2026": 41, "\u044b": 42, "b": 43, "x": 44, "\u043e": 45, "\u044d": 46, "\u044a": 47, "p": 48, "\u0430": 49, "\u043f": 50, "\u044e": 51, "\u2212": 52, "\u0435": 53, "\u0432": 54, "\u044c": 55, "r": 57, "t": 58, "\u043a": 59, ")": 60, "f": 61, "n": 62, "|": 56, "[UNK]": 63, "[PAD]": 64}
with open('vocab1.json', 'w') as vocab_file:
    json.dump(vocab_dict1, vocab_file)

In [15]:
vocab_dict1

{'m': 0,
 'ё': 1,
 'e': 2,
 'i': 3,
 'ж': 4,
 '‑': 5,
 's': 6,
 'я': 7,
 'р': 8,
 'м': 9,
 'н': 10,
 '«': 11,
 'й': 12,
 'g': 13,
 'т': 14,
 '–': 15,
 'k': 16,
 'z': 17,
 '—': 18,
 'з': 19,
 "'": 20,
 'a': 21,
 'д': 22,
 'л': 23,
 '»': 24,
 'ч': 25,
 'с': 26,
 'б': 27,
 'h': 28,
 'c': 29,
 '(': 30,
 'и': 31,
 'l': 32,
 'щ': 33,
 'ф': 34,
 'o': 35,
 'ш': 36,
 'у': 37,
 'х': 38,
 'г': 39,
 'ц': 40,
 '…': 41,
 'ы': 42,
 'b': 43,
 'x': 44,
 'о': 45,
 'э': 46,
 'ъ': 47,
 'p': 48,
 'а': 49,
 'п': 50,
 'ю': 51,
 '−': 52,
 'е': 53,
 'в': 54,
 'ь': 55,
 'r': 57,
 't': 58,
 'к': 59,
 ')': 60,
 'f': 61,
 'n': 62,
 '|': 56,
 '[UNK]': 63,
 '[PAD]': 64}

In [16]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/kaggle/working/vocab1.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [17]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [18]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [19]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [23]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [24]:
import torch

In [25]:
torch.__version__

'1.11.0'

In [26]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=1)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=1)

  0%|          | 0/23444 [00:00<?, ?ex/s]

  0%|          | 0/8007 [00:00<?, ?ex/s]

In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [28]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [29]:
wer_metric = load_metric("wer")

In [30]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [31]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "/kaggle/working/pretrained/model", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [32]:
model.freeze_feature_extractor()

/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1619: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [33]:
model.gradient_checkpointing_enable()

batch_size=20- загрузка 10гб первая карта

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-basee-ru-demo",
  group_by_length=True,
  per_device_train_batch_size=25,
  gradient_accumulation_steps=16,
  evaluation_strategy="steps",
  num_train_epochs=100,
  fp16=True,
  save_steps=50,
  eval_steps=25,
  logging_steps=10,
  learning_rate=1e-4,
  #warmup_steps=500,
  save_total_limit=1,
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train.with_format("torch"),
    eval_dataset=common_voice_test.with_format("torch"),
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [40]:
import os 

os.environ["WANDB_DISABLED"] = "true"

In [37]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 23444
  Num Epochs = 100
  Instantaneous batch size per device = 25
  Total train batch size (w. parallel, distributed & accumulation) = 400
  Gradient Accumulation steps = 16
  Total optimization steps = 5800
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Wer
25,0.233700,0.507422,0.495476
50,0.224900,0.511023,0.498443
75,0.215000,0.498942,0.491266
100,0.238500,0.504927,0.493404
125,0.251400,0.505427,0.489582
150,0.227300,0.495714,0.491894
175,0.233400,0.496664,0.486975
200,0.229400,0.493352,0.492602
225,0.242000,0.497976,0.486855
250,0.239100,0.486771,0.479237


***** Running Evaluation *****
  Num examples = 8007
  Batch size = 8
***** Running Evaluation *****
  Num examples = 8007
  Batch size = 8
Saving model checkpoint to ./wav2vec2-basee-ru-demo/checkpoint-50
Configuration saved in ./wav2vec2-basee-ru-demo/checkpoint-50/config.json
Model weights saved in ./wav2vec2-basee-ru-demo/checkpoint-50/pytorch_model.bin
Feature extractor saved in ./wav2vec2-basee-ru-demo/checkpoint-50/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 8007
  Batch size = 8
***** Running Evaluation *****
  Num examples = 8007
  Batch size = 8
Saving model checkpoint to ./wav2vec2-basee-ru-demo/checkpoint-100
Configuration saved in ./wav2vec2-basee-ru-demo/checkpoint-100/config.json
Model weights saved in ./wav2vec2-basee-ru-demo/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./wav2vec2-basee-ru-demo/checkpoint-100/preprocessor_config.json
Deleting older checkpoint [wav2vec2-basee-ru-demo/checkpoint-50] due to args.save_total_limit


KeyboardInterrupt: 

Посмотрим, насколько модель обучилась (кажется, что не очень)

In [38]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 8007
  Batch size = 8


{'eval_loss': 0.5310655236244202, 'eval_wer': 0.5006615966532565}

In [42]:
os.environ["WANDB_DISABLED"] = "false"

In [43]:
model_base = Wav2Vec2ForCTC.from_pretrained(
    "/kaggle/working/pretrained/model", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

trainer_base = Trainer(
    model=model_base,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train.with_format("torch"),
    eval_dataset=common_voice_test.with_format("torch"),
    tokenizer=processor.feature_extractor,
)

trainer_base.evaluate()

loading configuration file /kaggle/working/pretrained/model/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "./wav2vec2-basee-bachkir-demo/checkpoint-200",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "group",
  "fea

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 0.49487435817718506,
 'eval_wer': 0.4877237065451289,
 'eval_runtime': 237.2569,
 'eval_samples_per_second': 33.748,
 'eval_steps_per_second': 4.219}

In [46]:
torch.save({
        'model_state_dict': model.state_dict()
}, 'my_model.pth')